In [1]:
%pip install --upgrade openai
%pip install --upgrade python-dotenv
%pip install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.1/606.1 kB 20.1 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not write

In [ ]:
import openai
import os
from IPython.display import display, Markdown, Latex
from langchain.llms import OpenAI
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

openai.api_key  = '请将这里替换成你的API Key'


In [25]:
import os
import time
import logging
import openai
from typing import Collection

class BaseLLMModel:
    def __init__(self):
        pass

    def call(self, prompt):
        raise NotImplementedError("need to complete!")
    

# 定义调用 GPT 的子类
class GPTModel(BaseLLMModel):
    def __init__(self, api_key, model="gpt-3.5-turbo"):
        super().__init__()
        self.api_key = api_key
        self.system_prompt= """
        You are a state-of-the-art python code completion system that will be used as part of a genetic algorithm to evolve cap sets.
        You will be given a list of functions, and you should improve the incomplete last function in the list.
        1. Make only small changes but be sure to make some change.
        2. Try to keep the code short and any comments concise.
        3. Your response should be an implementation of the function function_to_evolve_v# (where # is the current iteration number); do not include any examples or extraneous functions.
        4. You may use numpy and itertools.
        The code you generate will be appended to the user prompt and run as a python program.
        """
        BASE_URL = 'https://api.bltcy.ai/v1'
        self.client = OpenAI(api_key=api_key, base_url=BASE_URL)
        self.model = model

    def call(self, prompt):
        content = f"${self.system_prompt} ${prompt}"
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": content}
                ],
                stream=False
            )
            print(response.choices[0].message.content)
        except Exception as e:
            print(f"Error calling GPT: {e}")
            return None
        
class LLM:
  """Language model that predicts continuation of provided source code."""

  def __init__(self, samples_per_prompt: int, model:BaseLLMModel) -> None:
    # define the prompt pool
    self._model = model
    self._samples_per_prompt = samples_per_prompt

  def _draw_sample(self, prompt: str) -> str:
    """Returns a predicted continuation of `prompt`."""
    return self._model.call(prompt)
    
  def draw_samples(self, prompt: str) -> Collection[str]:
    """Returns multiple predicted continuations of `prompt`."""
    return [self._draw_sample(prompt) for _ in range(self._samples_per_prompt)]


In [22]:
# 加载 .env 文件中的环境变量
from dotenv import load_dotenv

load_dotenv()

# 读取环境变量
openai_api_key = os.getenv('OPENAI_API_KEY')
gptModel = GPTModel(openai_api_key)

In [26]:
def read_specification_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"文件 {file_path} 未找到。")
        return None
test_prompt = read_specification_file('test_spec.txt')
llm = LLM(2,gptModel)
llm.draw_samples(test_prompt)


Error calling GPT: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}
Error calling GPT: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}


[None, None]